In [1]:
!pip install classiq torchinfo tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully 

In [4]:
import classiq

classiq.authenticate()

/usr/local/lib/python3.10/dist-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [5]:
"""_summary_
Linear Entanglement Quantum Model for MNIST Data Classification with three linear entanglement layers of RXX, RYY, and RZZ.
"""

from classiq import create_model, QFunc, QArray, QBit, Output, allocate, RX, RY, RZ, RZZ, RXX, RYY, CZ

@QFunc
def encoding(q: QArray[QBit]) -> None:
    """
    This function encodes the input data into the qubits. This input data is a 4x4 image pixel values
    converted into angle for rotation gates (RX, RY, RZ, RX) in form of a 16x1 vector.
    We encode 4 pixels per qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to encode the input data into.
    """
    RX(theta="input_0", target=q[0]) # Pixel 0 on Qubit 0
    RY(theta="input_1", target=q[0]) # Pixel 1 on Qubit 0
    RZ(theta="input_2", target=q[0]) # Pixel 2 on Qubit 0
    RX(theta="input_3", target=q[0]) # Pixel 3 on Qubit 0

    RX(theta="input_4", target=q[1]) # Pixel 4 on Qubit 1
    RY(theta="input_5", target=q[1]) # Pixel 5 on Qubit 1
    RZ(theta="input_6", target=q[1]) # Pixel 6 on Qubit 1
    RX(theta="input_7", target=q[1]) # Pixel 7 on Qubit 1

    RX(theta="input_8", target=q[2]) # Pixel 8 on Qubit 2
    RY(theta="input_9", target=q[2]) # Pixel 9 on Qubit 2
    RZ(theta="input_10", target=q[2]) # Pixel 10 on Qubit 2
    RX(theta="input_11", target=q[2]) # Pixel 11 on Qubit 2

    RX(theta="input_12", target=q[3]) # Pixel 12 on Qubit 3
    RY(theta="input_13", target=q[3]) # Pixel 13 on Qubit 3
    RZ(theta="input_14", target=q[3]) # Pixel 14 on Qubit 3
    RX(theta="input_15", target=q[3]) # Pixel 15 on Qubit 3

@QFunc
def mixing(q: QArray[QBit]) -> None:
    """
    This function performs the mixing operation on the qubits.
    This is done by applying a series of RZZ, RXX, RYY gates to form a
    ring connection.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the mixing operation on.
    """
    RZZ(theta="weight_0", target=q[0:2])
    RZZ(theta="weight_1", target=q[1:3])
    RZZ(theta="weight_2", target=q[2:4])

    RXX(theta="weight_4", target=q[0:2])
    RXX(theta="weight_5", target=q[1:3])
    RXX(theta="weight_6", target=q[2:4])

    RYY(theta="weight_8", target=q[0:2])
    RYY(theta="weight_9", target=q[1:3])
    RYY(theta="weight_10", target=q[2:4])

@QFunc
def cz_block(q: QArray[QBit]) -> None:
    """
    This function applies CZ gates between each qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the entanglement operation on.
    """
    CZ(control=q[0], target=q[1])
    CZ(control=q[1], target=q[2])
    CZ(control=q[2], target=q[3])

@QFunc
def main(res: Output[QArray[QBit]]) -> None:
    """
    This is the main function from which model will be created.
    It calls the other functions to perform the encoding, mixing and entanglement.

    Args:
        res (Output[QArray[QBit]]): Output QArray of QBits from which the model will be created.
    """
    allocate(4, res)
    encoding(q=res)
    mixing(q=res)
    cz_block(q=res)

def linear_entanglement_r3_quantum_model():
    model = create_model(main)
    return model

In [6]:
import torch

from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram
from classiq.applications.qnn import QLayer
from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

def execute_fn(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

def post_process_fn(result: SavedResult) -> torch.Tensor:
    counts: dict = result.value.counts

    # Calculate logits from counts
    logits: float = torch.zeros(16)
    for key, value in counts.items():
        logits[int(key, 2)] = value

    # Trim the logits from length 16 to length 10 since we have only 10 labels
    trimmed_logits = logits[:2]

    # Calculate prediction probabilities from logits by normalizing it
    pred_probs = torch.nn.functional.normalize(trimmed_logits, dim=0)

    # Convert the prediction probabilities into prediction labels
    pred_labels = torch.argmax(pred_probs)

    ### WRITE COUNTS, OUTPUT LOGITS, PRED PROBS, PRED LABELS to a file
    # output_file = open("post_process_output.txt", "a")
    # print("----------------------------------------------------------------------------------------------------------------------------------------------", file=output_file)
    # print(f"COUNTS:: \n {counts} \n", file=output_file)
    # print(f"LOGITS:: \n {logits} \n", file=output_file)
    # print(f"TRIMMED LOGITS:: \n {trimmed_logits} \n", file=output_file)
    # print(f"PREDICTION PROBABILITIES:: \n {pred_probs} \n", file=output_file)
    # print(f"PREDICTION LABELS:: \n {pred_labels} \n", file=output_file)
    # output_file.close()

    return pred_probs.clone().detach()

class QNN(torch.nn.Module):
    def __init__(self, quantum_program, execute, post_process, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute=execute,
            post_process=post_process,
            *args,
            **kwargs
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

In [7]:
import os
from typing import Callable,Optional

from torchvision import datasets
from torch.utils.data import DataLoader, Subset

NUM_WORKERS = os.cpu_count()

def create_dataloaders_from_folders(
    train_dir: str,
    test_dir: str,
    batch_size: int,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    num_workers: int = NUM_WORKERS
):
    """Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: function having torchvision transforms to perform on training and testing data.
        target_transform: function having torchvision transforms to perform on training and testing data labels.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
        train_dataloader, test_dataloader, class_names = \
            = create_dataloaders_from_folders(train_dir=path/to/train_dir,
                                            test_dir=path/to/test_dir,
                                            transform=some_data_transform_function,
                                            target_transform=some_label_transform_function,
                                            batch_size=32,
                                            num_workers=4)
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform, target_transform=target_transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform, target_transform=target_transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

def create_mnist_dataloaders(
    batch_size: int,
    root: str = "data",
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    num_workers: int = NUM_WORKERS,
    create_subset: bool = False,
    subset_size:int = 64
):
    """Creates training and testing DataLoaders.

    Creates PyTorch Dataloaders from PyTorch MNIST Dataset.

    Args:
        root: folder name in which data will be downloaded.
        transform: torchvision transforms to perform on training and testing data.
        target_transform: function having torchvision transforms to perform on training and testing data labels.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.
        create_subset: If True, it create a dataloaders from small subset of data.
        subset_size: Size of the subset of data. Defaults to 64.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
        train_dataloader, test_dataloader, class_names = \
            = create_mnist_dataloaders(root="data"
                                    transform=some_data_transform_function,
                                    target_transform=some_label_transform_function,
                                    batch_size=32,
                                    num_workers=4)
    """
    # Setup training data
    train_data = datasets.MNIST(
        root=root,
        train=True,
        download=True,
        transform=transform,
        target_transform=target_transform
    )

    # Setup testing data
    test_data = datasets.MNIST(
        root=root,
        train=False,
        download=True,
        transform=transform,
        target_transform=target_transform
    )

    # Get class names
    class_names = train_data.classes

    # Create subsets of the datasets
    if create_subset:
        train_data = Subset(train_data, range(subset_size))
        test_data = Subset(test_data, range(subset_size))

    # Turn datasets into iterables (batches)
    train_dataloader = DataLoader(train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )
    test_dataloader = DataLoader(test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return train_dataloader, test_dataloader, class_names

In [31]:
import torch
import torchvision.transforms as transforms

def input_transform(image):
    """
    The input MNIST images are all 28 × 28 px. This function will firstly center-crop
    them to 24 × 24 and then down-sample them to 4 × 4 for MNIST. Then we convert
    the image pixels into angles for passing them into Rotation gates later for encoding.
    """
    image = transforms.Grayscale(num_output_channels=1)(image)
    image = transforms.ToTensor()(image)
    image = transforms.CenterCrop(24)(image)
    image = transforms.Resize(size = (4,4), antialias=True)(image)
    image = image.squeeze()
    image_pixels = torch.flatten(image)
    angles = torch.sqrt(image_pixels / 256)

    return angles

def target_transform(label):
    label_tensor = torch.LongTensor([label])
    one_hot_label = torch.nn.functional.one_hot(label_tensor, 10)
    return one_hot_label.squeeze()

def target_transform_bin(label):
    label_tensor = torch.LongTensor([label])
    one_hot_label = torch.nn.functional.one_hot(label_tensor, 2)
    return one_hot_label.squeeze()

In [9]:
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os
import pandas as pd

def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/leqm3/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                                model_name="leqm3",
                                extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/leqm3/5_epochs/")
    """

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

def write_train_results(
    experiment_name,
    model_name,
    epochs,
    results,
):
    output_dir = "outputs/train_results/"

    # Check if the directory exists, and create it if not
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_file_name = f"{experiment_name}_{model_name}_epochs_{epochs}.csv"
    file_path = f"outputs/train_results/{output_file_name}"

    data = {
        'Epoch': list(range(1, len(results['train_loss']) + 1)),
        'Train Loss': results['train_loss']
    }

    df = pd.DataFrame(data)

    if os.path.exists(file_path):
        # If it exists, append data
        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        df.to_csv(file_path, mode='w', index=False, header=True)

In [10]:
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path


def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include
            either ".pth" or ".pt" as the file extension.

    Example usage:
        save_model(model=model_0,
                    target_dir="models",
                    model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(
        ".pt"
    ), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

def test(
    model: nn.Module,
    data_loader: DataLoader,
    atol=0,
    device: str = 'cpu',
) -> float:
    num_correct = 0
    total = 0

    # Put the model in eval mode
    model.eval()

    # Turn on inference mode context manager
    with torch.inference_mode():
        for data, labels in data_loader:
            # Send data to GPU
            data, labels = data.to(device), labels.to(device)

            # 1. Forward pass: Let the model predict
            predictions = model(data)

            # Get a tensor of booleans, indicating if each label is close to the real label
            is_prediction_correct = torch.isclose(predictions.argmax(dim=1), labels.argmax(dim=1), atol=atol)

            ### WRITE OUTPUT TO A FILE
            # output_file = open("test_loop_output.txt", "a")
            # print("----------------------------------------------------------------------------------------------------------------------------------------------", file=output_file)
            # print(f"LABELS:: \n {labels} \n", file=output_file)
            # print(f"PREDICTIONS:: \n {predictions} \n", file=output_file)
            # print(f"IS PREDICTIONS CORRECT:: \n {is_prediction_correct} \n", file=output_file)
            # output_file.close()

            # Count the amount of `True` predictions
            num_correct += is_prediction_correct.sum().item()

            # Count the total evaluations
            #   the first dimension of `labels` is `batch_size`
            total += labels.size(0)

    # Calculate the accuracy
    accuracy = float(num_correct) / float(total)
    print(f"Test Accuracy of the model: {accuracy * 100:.2f}%")
    return accuracy * 100

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from typing import Dict, List
from tqdm.auto import tqdm

def train(
    model: nn.Module,
    data_loader: DataLoader,
    loss_fn: nn.modules.loss._Loss,
    optimizer: optim.Optimizer,
    writer: torch.utils.tensorboard.writer.SummaryWriter,
    epochs: int = 20,
    device: str = 'cpu',
) -> Dict[str, List]:
    model.to(device)

    # Setup train loss value
    train_loss = 0

    # Create empty results dictionary
    results = {
        "train_loss": [],
    }

    # Loop through training steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n----------")
        for batch, (data, label) in enumerate(data_loader):
            # Send data to device (GPU or CPU)
            data, label = data.to(device), label.to(device)

            # 1. Forward pass
            output = model(data).to(device)

            # 2. Calculate loss
            loss = loss_fn(output, label)
            train_loss += loss

            # 3. Optimizer zero grad
            optimizer.zero_grad()

            # 4. Loss backward
            loss.backward()

            # 5. Optimizer step
            optimizer.step()

        # Calculate loss per epoch and print out what's happening
        train_loss /= len(data_loader)

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"Train loss: {train_loss:.5f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss.detach().item())

        ### Experiment Tracking ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add loss results to SummaryWriter
            writer.add_scalars(
                main_tag="Loss",
                tag_scalar_dict={"train_loss": train_loss,},
                global_step=epoch
            )

            # Close the writer
            writer.close()

    # Return the filled results at the end of the epochs
    return results

### Imports and Setup

In [13]:
import torch
import classiq

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

import sys
sys.path.append("../..") # Add the parent directory to the sys.path list

# from models.leqm3 import linear_entanglement_r3_quantum_model
# from models.qnn import execute_fn, post_process_fn, QNN

# from scripts.helper import create_writer, write_train_results
# from scripts.data_setup import create_mnist_dataloaders
# from scripts.data_transforms import input_transform, target_transform
# from scripts.train import train
# from scripts.test import test
# from scripts.save_model import save_model

In [14]:
## Authenticate Classiq
# classiq.authenticate()

In [15]:
## For setting up device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
device

'cpu'

In [16]:
# ## Clear Output Files
# post_process_output_file = open("post_process_output.txt", "w")
# print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
# print("--------------------------------------------POST PROCESS OUTPUT--------------------------------------------------", file=post_process_output_file)
# print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
# post_process_output_file.close()

# test_loop_output_file = open("test_loop_output.txt", "w")
# print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
# print("-----------------------------------------------TEST LOOP OUTPUT--------------------------------------------------", file=test_loop_output_file)
# print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
# test_loop_output_file.close()

In [17]:
## HYPER PARAMETERS
_LEARNING_RATE = 1.0
BATCH_SIZE = 32
EPOCHS = 10

### Quantum Model

In [18]:
## Create a Linear Entanglement Quantum Model for MNIST Data Classification with three linear entanglement layers of RXX, RYY, and RZZ.
quantum_model = linear_entanglement_r3_quantum_model()

In [19]:
quantum_program = classiq.synthesize(quantum_model)

In [20]:
# View Quantum Program on Classiq Platform
# classiq.show(quantum_program)

### Quantum Neural Network

In [21]:
qnn = QNN(
    quantum_program=quantum_program,
    execute=execute_fn,
    post_process=post_process_fn,
)

In [22]:
summary(model=qnn, input_size=(32, 16), verbose=0, col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20, row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
QNN (QNN)                                [32, 16]             [32, 2]              --                   True
├─QLayer (qlayer)                        [32, 16]             [32, 2]              9                    True
Total params: 9
Trainable params: 9
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [23]:
# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

### Preparing Data

In [24]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "mini_data_1280_bin"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "mini_data_1280_bin.zip", "wb") as f:
    request = requests.get("https://github.com/devilkiller-ag/QNN-MNIST-Classification/raw/main/mini_data_1280_bin.zip")
    print("Downloading mnist bin data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "mini_data_1280_bin.zip", "r") as zip_ref:
    print("Unzipping mnist bin data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "mini_data_1280_bin.zip")

data/mini_data_1280_bin directory exists.
Unzipping mnist bin data...


In [25]:
from pathlib import Path
train_dir = Path('data/mini_data_1280_bin/mini_data_1280_bin/train')
test_dir = Path('data/mini_data_1280_bin/mini_data_1280_bin/test')

In [35]:
train_dataloader, test_dataloader, class_names = create_dataloaders_from_folders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=input_transform,
    target_transform=target_transform_bin,
    batch_size=BATCH_SIZE,
)

In [36]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")
print(f"Our Dataset have following classes: {class_names}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x793d18718520>, <torch.utils.data.dataloader.DataLoader object at 0x793c70ee6710>)
Length of train dataloader: 80 batches of 32
Length of test dataloader: 80 batches of 32
Our Dataset have following classes: ['0', '1']


In [37]:
data, label = next(iter(train_dataloader))

print(f"Image shape: {data.shape} -> [batch_size, pixel_angle]")
print(f"Label shape: {label.shape} -> [batch_size, label_value]")

Image shape: torch.Size([32, 16]) -> [batch_size, pixel_angle]
Label shape: torch.Size([32, 2]) -> [batch_size, label_value]


#### Run Experiment

##### 01. Train

In [29]:
# Create a writer for tracking our experiment
writer = create_writer(experiment_name="custom_data_1280", model_name="linear_entanglement_r3", extra=f"{EPOCHS}_epochs")

[INFO] Created SummaryWriter, saving to: runs/2024-01-08/custom_data_1280/linear_entanglement_r3/10_epochs...


In [ ]:
train_results = train(
    model = qnn,
    data_loader = train_dataloader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    writer = writer,
    epochs = EPOCHS,
    device = device
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
----------


In [ ]:
# Check out the model results
print(train_results)

In [ ]:
write_train_results(experiment_name="data_1280_bin", model_name="linear_entanglement_r3", epochs=EPOCHS, results=train_results)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs

##### 02. Save the trained model

In [ ]:
save_model(
    model=qnn,
    target_dir='outputs/saved_models',
    model_name=f'exp_1_leqmr3_data_1280_bin_epoch{EPOCHS}.pt'
)

##### 03. Test

In [ ]:
test_results = test(
    model = qnn,
    data_loader = test_dataloader,
    device = device
)

In [ ]:
print(test_results)